In [ ]:
 !pip install simpletransformers
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 13.3 MB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import emoji
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
# Use NLTK to preprocess the data
nltk.download('stopwords')
nltk.download('wordnet') # used for lemmatization
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd

base_file_path = '/content/drive/My Drive/ML4NLP/ex05/data/'

train_data_path = base_file_path + 'EI-reg-En-anger-train.txt'
test_data_path = base_file_path + '2018-EI-reg-En-anger-test-gold.txt'

train_df = pd.read_csv(train_data_path, delimiter='\t')

test_df = pd.read_csv(test_data_path, delimiter='\t')

### Correct first row
def fix_first_dataframe_row(df):
    """
    Split the first row's column values and assign them to the correct columns
    Shift everything to the right by one column.
    """
    fist_id_column_values = df.at[0, 'ID'].split('   ')
    df.at[0, 'Intensity Score'] = float(df.at[0, 'Affect Dimension'])
    df.at[0, 'Affect Dimension'] = df.at[0, 'Tweet']
    df.at[0, 'Tweet'] = fist_id_column_values[1]
    df.at[0, 'ID'] = fist_id_column_values[0]

fix_first_dataframe_row(train_df)
train_df.head()

ID                                              Tweet  \
0  2017-En-10264  @xandraaa5 @amayaallyn6 shut up hashtags are c...   
1  2017-En-10072  it makes me so fucking irate jesus. nobody is ...   
2  2017-En-11383         Lol Adam the Bull with his fake outrage...   
3  2017-En-11102  @THATSSHAWTYLO passed away early this morning ...   
4  2017-En-11506  @Kristiann1125 lol wow i was gonna say really?...   

  Affect Dimension  Intensity Score  
0            anger            0.562  
1            anger            0.750  
2            anger            0.417  
3            anger            0.354  
4            anger            0.438

In [ ]:
## PREPROCESSING
import string

punctuation = string.punctuation.replace('@', '')
mention_pattern = r'@\w+'

def preprocess_text(text: str) -> str:
    """
    Preprocess the text by converting to lowercase, removing punctuation, URLs, emojis, and newlines.
    """
    text = text.lower()
    # unify puntctuation
    text = text.translate(str.maketrans('', '', punctuation))
    text = text.strip()

    #anonymize mentions
    text = re.sub(mention_pattern, '<MENTION>', text)

    # convert emojis to words
    text = emoji.demojize(text, delimiters=("", ""))
    # remove URLs
    text = re.sub(r'http\S+', '', text)
    # replace newline characters with space
    text = text.replace('\n', ' ')
    return text

In [ ]:
# Preprocess the data
train_df['Tweet'] = train_df['Tweet'].apply(preprocess_text)
test_df['Tweet'] = test_df['Tweet'].apply(preprocess_text)
train_df['Tweet'][:5]

0    <MENTION> <MENTION> shut up hashtags are cool ...
1    it makes me so fucking irate jesus nobody is c...
2              lol adam the bull with his fake outrage
3    <MENTION> passed away early this morning in a ...
4    <MENTION> lol wow i was gonna say really haha ...
Name: Tweet, dtype: object

In [ ]:
def rename_columns_and_drop_others(df):
    """
    Rename the columns to match the expected input of the model.
    Drop the columns that are not needed.
    """
    df.rename(columns={'Tweet': 'text' , 'Intensity Score': 'labels'}, inplace=True)
    df.drop(columns=['Affect Dimension', 'ID'], inplace=True)


rename_columns_and_drop_others(train_df)
rename_columns_and_drop_others(test_df)

train_df.head()

text  labels
0  <MENTION> <MENTION> shut up hashtags are cool ...   0.562
1  it makes me so fucking irate jesus nobody is c...   0.750
2            lol adam the bull with his fake outrage   0.417
3  <MENTION> passed away early this morning in a ...   0.354
4  <MENTION> lol wow i was gonna say really haha ...   0.438

In [ ]:
type(train_df['labels'][0])

numpy.float64

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs(num_train_epochs=1, manual_seed=42, train_batch_size=4, max_seq_length=128)
model_args.num_train_epochs = 3
model_args.regression = True
model_args.overwrite_output_dir = True

models = {'distilbert': 'distilbert-base-uncased', 'roberta': 'roberta-base', 'bert': 'bert-base-uncased'}

def get_model(model_kind: str, model_name: str) -> ClassificationModel:
  # Create a ClassificationModel
  return ClassificationModel(
      model_kind,
      model_name,
      num_labels=1,
      args=model_args,
      use_cuda=True if torch.cuda.is_available() else False
  )

In [ ]:
from scipy.stats import pearsonr

def train_and_evaluate_models():
  # train
  for model_kind, model_name in models.items():
    model = get_model(model_kind, model_name)
    model.train_model(train_df)
    result, model_outputs, wrong_predictions = model.eval_model(test_df)

    # evaulate model based on pearson r coefficient
    pearson_r = pearsonr(model_outputs, test_df['labels'])



    print(f"Result for {model_name}")
    print(f'Mean Squared Error: {result["eval_loss"]}')
    print(f'Pearson Coefficient: {pearson_r}')

In [ ]:
train_and_evaluate_models()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1701 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/426 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/426 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/426 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/426 [00:00<?, ?it/s]

  0%|          | 0/1002 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/126 [00:00<?, ?it/s]

Result for bert-base-uncased
Mean Squared Error: 0.013716535136428855
Pearson Coefficient: PearsonRResult(statistic=0.7919854416655973, pvalue=1.6480149394555955e-216)
